### Use Word2Vec to train your own model on a dataset.

1) **Optional** - Find your own dataset of documents to train you model on. You are going to need a lot of data, so it's probably not realistic to scrape data for this assignment given the time constraints that we're working under. Try to find a dataset that has > 5000 documents.

- If you can't find a dataset to use try this one: <https://www.kaggle.com/c/quora-question-pairs>

2) Clean/Tokenize the documents.

3) Vectorize the model using Word2Vec and explore the results using each of the following at least one time:

- your_model.wv.most_similar()
- your_model.wv.similarity()
- your_model.wv.doesn't_match()

In [1]:
import pandas as pd
url = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-2-NLP/master/module4-Word-Embeddings/simpsons_dataset.csv'
df = pd.read_csv(url)

In [19]:
import string
from nltk.tokenize import word_tokenize
def process_text(text):
  """Remove punctuation, lowercase, and tokenize text."""
  # TODO: check for special cases like "I'll"
  text = "".join([char.lower() for char in text
                  if char not in string.punctuation])
  return word_tokenize(text)

In [20]:
df['homer'] = (df.raw_character_text == 'Homer Simpson')
# df['homer'] = (df.raw_character_text.map({'Homer Simpson':1}))
df.head(5)

,raw_character_text,spoken_words,homer
0,Miss Hoover,"No, actually, it was a little of both. Sometim...",False
1,Lisa Simpson,Where's Mr. Bergstrom?,False
2,Miss Hoover,I don't know. Although I'd sure like to talk t...,False
3,Lisa Simpson,That life is worth living.,False
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...,False


In [21]:
df.raw_character_text.value_counts()[:5]

Homer Simpson          29782
Marge Simpson          14141
Bart Simpson           13759
Lisa Simpson           11489
C. Montgomery Burns     3162
Name: raw_character_text, dtype: int64

In [22]:
df.tokens = df.spoken_words.astype('str').apply(process_text)

/home/superio/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [23]:
!pip install -U gensim
import gensim

    100% |████████████████████████████████| 24.2MB 628kB/s ta 0:00:011 3% |█▏                              | 901kB 5.1MB/s eta 0:00:05    5% |█▊                              | 1.3MB 1.6MB/s eta 0:00:15    7% |██▌                             | 1.9MB 2.9MB/s eta 0:00:08    48% |███████████████▌                | 11.7MB 4.7MB/s eta 0:00:03    61% |███████████████████▌            | 14.8MB 4.3MB/s eta 0:00:03    69% |██████████████████████▏         | 16.8MB 5.4MB/s eta 0:00:02
    100% |████████████████████████████████| 61kB 636kB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 767kB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 668kB/s ta 0:00:01
    100% |████████████████████████████████| 5.4MB 2.1MB/s ta 0:00:01
  Stored in directory: /home/superio/.cache/pip/wheels/b8/cb/43/c0ba52baf2b0e371ec1d5b2d4685d6d24617b1391f3eeacda5
Successfully built smart-open


In [24]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/superio/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/superio/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/superio/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/superio/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/superio/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/superio/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to
[nltk_data]    |     /home/su

[nltk_data]    |   Unzipping corpora/sentence_polarity.zip.
[nltk_data]    | Downloading package shakespeare to
[nltk_data]    |     /home/superio/nltk_data...
[nltk_data]    |   Unzipping corpora/shakespeare.zip.
[nltk_data]    | Downloading package sinica_treebank to
[nltk_data]    |     /home/superio/nltk_data...
[nltk_data]    |   Unzipping corpora/sinica_treebank.zip.
[nltk_data]    | Downloading package smultron to
[nltk_data]    |     /home/superio/nltk_data...
[nltk_data]    |   Unzipping corpora/smultron.zip.
[nltk_data]    | Downloading package state_union to
[nltk_data]    |     /home/superio/nltk_data...
[nltk_data]    |   Unzipping corpora/state_union.zip.
[nltk_data]    | Downloading package stopwords to
[nltk_data]    |     /home/superio/nltk_data...
[nltk_data]    |   Package stopwords is already up-to-date!
[nltk_data]    | Downloading package subjectivity to
[nltk_data]    |     /home/superio/nltk_data...
[nltk_data]    |   Unzipping corpora/subjectivity.zip.
[nltk_da

True

In [25]:
import pandas as pd
from gensim.models.word2vec import Word2Vec
from nltk.tokenize import word_tokenize

import string

def process_text(text):
  """Remove punctuation, lowercase, and tokenize text."""
  # TODO: check for special cases like "I'll"
  text = "".join([char.lower() for char in text
                  if char not in string.punctuation])
  return word_tokenize(text)

#sentences = [process_text(X_train['question2'][3])]
sentences = [process_text(str(i)) for i in df.spoken_words]

In [26]:
print(len(sentences))
model = Word2Vec(sentences, min_count=1, size=5)
print(model)
print(list(model.wv.vocab))
print(len(model.wv.vocab))

158314
Word2Vec(vocab=45518, size=5, alpha=0.025)
['no', 'actually', 'it', 'was', 'a', 'little', 'of', 'both', 'sometimes', 'when', 'disease', 'is', 'in', 'all', 'the', 'magazines', 'and', 'news', 'shows', 'its', 'only', 'natural', 'that', 'you', 'think', 'have', 'wheres', 'mr', 'bergstrom', 'i', 'dont', 'know', 'although', 'id', 'sure', 'like', 'to', 'talk', 'him', 'he', 'didnt', 'touch', 'my', 'lesson', 'plan', 'what', 'did', 'teach', 'life', 'worth', 'living', 'polls', 'will', 'be', 'open', 'from', 'now', 'until', 'end', 'recess', 'just', 'case', 'any', 'decided', 'put', 'thought', 'into', 'this', 'well', 'our', 'final', 'statements', 'martin', 'theres', 'anything', 'left', 'say', 'bart', 'victory', 'party', 'under', 'slide', 'nan', 'hey', 'moved', 'out', 'morning', 'must', 'new', 'job', 'took', 'his', 'copernicus', 'costume', 'do', 'where', 'could', 'find', 'hes', 'taking', 'next', 'train', 'capital', 'city', 'how', 'traditional', 'yet', 'environmentally', 'sound', 'yes', 'been', '

In [28]:
model.wv.similarity("smarter","ponies")

0.6513287

In [27]:
model.wv.most_similar("haha")

[('wiser', 0.9989606142044067),
 ('waiwaiwait', 0.9988879561424255),
 ('bakeoff', 0.9984931945800781),
 ('tarragoncrusted', 0.9984617829322815),
 ('robe', 0.9983481168746948),
 ('muscle', 0.9983134269714355),
 ('completing', 0.9982211589813232),
 ('badgers', 0.9982205033302307),
 ('fatherson', 0.9981105923652649),
 ('pirate', 0.9979286193847656)]

In [32]:
dir(model.wv)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_load_specials',
 '_log_evaluate_word_analogies',
 '_save_specials',
 '_smart_save',
 'accuracy',
 'add',
 'closer_than',
 'cosine_similarities',
 'distance',
 'distances',
 'doesnt_match',
 'evaluate_word_analogies',
 'evaluate_word_pairs',
 'get_keras_embedding',
 'get_vector',
 'index2entity',
 'index2word',
 'init_sims',
 'load',
 'load_word2vec_format',
 'log_accuracy',
 'log_evaluate_word_pairs',
 'most_similar',
 'most_similar_cosmul',
 'most_similar_to_given',
 'n_similarity',
 'rank',
 'relative_cosine_similarity',
 'save',
 'save_word2vec_for

### Stretch Goals:

1) Use Doc2Vec to train a model on your dataset, and then provide model with a new document and let it find similar documents.

2) Download the pre-trained word vectors from Google. Access the pre-trained vectors via the following link: https://code.google.com/archive/p/word2vec

Load the pre-trained word vectors and train the Word2vec model

Examine the first 100 keys or words of the vocabulary

Outputs the vector representation for a select set of words - the words can be of your choice

Examine the similarity between words - the words can be of your choice

For example:

model.similarity('house', 'bungalow')

model.similarity('house', 'umbrella')